In [7]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [8]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

'''
# Initialize the Sequential model
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4)

# Save the model
model.save('textgenerator.model')

'''

1115394/1115394 [==============================] - 0s 0us/step


<ipython-input-8-dd5987b4544e>:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
<ipython-input-8-dd5987b4544e>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(characters)), dtype=np.bool)


Epoch 1/4
651/651 [==============================] - 117s 177ms/step - loss: 2.7092
Epoch 2/4
651/651 [==============================] - 112s 171ms/step - loss: 2.3005
Epoch 3/4
651/651 [==============================] - 112s 172ms/step - loss: 2.1810
Epoch 4/4
651/651 [==============================] - 111s 171ms/step - loss: 2.0958


In [9]:
model= tf.keras.models.load_model('textgenerator.model')

In [10]:
def sample(preds, temperature= 1.0):
  preds= np.asarray(preds).astype('float64')
  preds= np.log(preds)/temperature
  exp_preds= np.exp(preds)
  preds= exp_preds/ np.sum(exp_preds)
  probas= np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [20]:
def generate_text(length, temperature):
  start_index= random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated=''
  sentence= text[start_index : start_index + SEQ_LENGTH]
  generated+= sentence
  for i in range(length):
    x= np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      x[0, t, char_to_index[character]]= 1

    prediction= model.predict(x, verbose=0)[0]
    next_index= sample(prediction, temperature)
    next_character= index_to_char[next_index]

    generated+= next_character
    sentence = sentence[1:] + next_character
  return generated


In [21]:
print('---------0.2---------')
print(generate_text(300, 0.2))
print('---------0.4---------')
print(generate_text(300, 0.4))
print('---------0.6---------')
print(generate_text(300, 0.6))
print('---------0.8---------')
print(generate_text(300, 0.8))
print('---------1---------')
print(generate_text(300, 1.0))


---------0.2---------
kill'd, he's dead!

juliet:
can heaven beat the head the sore the have the warte the seather and the with the parte the sare to the bute the are the and and and and and mand the beather in the sond and and and the mear and and the pare the sore to the pare the seall of the his the the meres and and and the will the proust the the the with
---------0.4---------
er
and the lord hastings, who attended have the hard and hear hat ay his for thae and and at in the for the cour the fore hourd.

pardowe:
whith the mare the hare the each the allous seast
and betere in this beather counted i hand and this the ming casten steed and of mere the deart of and nower of mear the preast the pare the have the wo
---------0.6---------
our heart for sending me about,
to catch sere hest yous in thy beantstale and the porting and then?
browi:
shee, the wanco dusest the undentore's then,
to the way hames, and benter and the comas it the ard of out here cous ford fearind thou sof in ard 